1. Завантажити та попередньо обробити дані твітів за допомогою функції process_tweet.

In [ ]:
import nltk
nltk.download("twitter_samples")

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [5]:
from nltk.corpus import twitter_samples
import numpy as np


In [6]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))


In [7]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

In [9]:
def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    #tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


2. Побудувати словник частотності слів для позитивного і негативного класів.


In [11]:
def count_tweets(result, tweets, ys):
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)

            if pair in result:
                result[pair] += 1
            else:
                result[pair] = 1

    return result

In [12]:
freqs = count_tweets({}, train_x, train_y)

In [26]:
freqs[('fuck', 1)]

20

In [25]:
freqs[('fuck', 0)]

47

In [14]:
len(freqs)

11397

3. Обчислити логарифмічні апріорні ймовірності (log prior) для кожного класу.

Алгоритм:
- Порахувати кількість унікальних слів V.
- Порахувати загальну кількість позитивних слів N_pos і негативних слів N_neg.
- Порахувати кількість документів D.
- Порахувати кількість позитивних документів D_pos і негативних документів D_neg.
- Обчислити log prior:
    logprior = log(D_pos) - log(D_neg)
- Для кожного слова у словнику:
    - Дістати частотності слова для позитивного і негативного класів.
    - Обчислити ймовірності слова для кожного класу з лапласівським згладжуванням:
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)  
    - Обчислити log likelihood як логарифм відношення ймовірностей:  
        loglikelihood = log(p_w_pos / p_w_neg)

Можлива реалізація:

4. Обчислити логарифмічні правдоподібності (log likelihood) для кожного слова і класу.



In [27]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0

    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    N_pos = N_neg = 0
    for pair in freqs.keys():
        if pair[1] > 0:
            N_pos += freqs[pair]
        else:
            N_neg += freqs[pair]

    D = len(train_y)
    D_pos = sum(train_y)
    D_neg = D - D_pos

    logprior = np.log(D_pos) - np.log(D_neg)

    for word in vocab:
        freq_pos = freqs.get((word, 1), 0)
        freq_neg = freqs.get((word, 0), 0)

        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        loglikelihood[word] = np.log(p_w_pos) - np.log(p_w_neg)

    return logprior, loglikelihood

In [29]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)

In [30]:
logprior

np.float64(0.0)

In [31]:
loglikelihood

{'hah': np.float64(0.6787947231732101),
 '61': np.float64(-0.7074996379466807),
 'episod': np.float64(-0.5251780811527276),
 'jai': np.float64(-0.014352457386735296),
 'paralyz': np.float64(0.6787947231732101),
 'epic': np.float64(-0.014352457386735296),
 'diva': np.float64(0.6787947231732101),
 'noseble': np.float64(-0.7074996379466807),
 'oliv': np.float64(0.6787947231732101),
 'knw': np.float64(-0.014352457386735296),
 '150': np.float64(0.6787947231732101),
 'airport': np.float64(-0.9951817103984624),
 'foood': np.float64(-0.7074996379466807),
 'gosh': np.float64(-0.5251780811527276),
 'raheel': np.float64(0.6787947231732101),
 'ngeze': np.float64(-0.7074996379466807),
 'islamist': np.float64(0.6787947231732101),
 'notic': np.float64(-0.5449807084489064),
 'charg': np.float64(-0.302034529838517),
 'myungsoo': np.float64(-0.7074996379466807),
 'cranium': np.float64(0.6787947231732101),
 'custom': np.float64(0.9019382744874189),
 'iran': np.float64(-0.7074996379466807),
 'dentist': np

5. Реалізувати функцію наївного баєсового класифікатора.  

Алгоритм:
- Обробити твіт з допомогою process_tweet.
- Встановити початкову ймовірність p = logprior.  
- Для кожного слова у твіті:
    - Якщо слово є в словнику loglikelihood:
        - Додати loglikelihood цього слова до p.
- Повернути суму ймовірностей p.

Приклад реалізації:




In [28]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    word_l = process_tweet(tweet)

    p = logprior

    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]

    return p

6. Оцінити точність класифікатора на тестовому наборі даних.

Алгоритм:
- Для кожного твіту в тестовому наборі:
    - Класифікувати твіт з допомогою naive_bayes_predict.
    - Якщо ймовірність > 0, присвоїти твіту мітку 1, інакше 0.
    - Додати передбачену мітку до списку.
- Порахувати середню абсолютну похибку між передбаченими та справжніми мітками.  
- Обчислити точність як 1 - похибка.

Приклад реалізації:


In [32]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    accuracy = 0

    y_hats = []
    for tweet in test_x:
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)

    error = np.mean(np.absolute(y_hats - test_y))
    accuracy = 1 - error

    return accuracy

In [33]:
accuracy = test_naive_bayes(test_x, test_y, logprior, loglikelihood)

In [34]:
accuracy

np.float64(0.9955)

7. Проаналізувати найбільш позитивні та негативні слова за співвідношенням їх частотностей.

Функція get_ratio обчислює відношення частотності слова для позитивного класу до негативного.

Алгоритм get_ratio:
- Ініціалізувати словник з ключами 'positive', 'negative', 'ratio' і значеннями 0.
- Знайти кількість появ слова для позитивного класу з допомогою функції lookup і записати в 'positive'.
- Знайти кількість появ слова для негативного класу і записати в 'negative'.
- Обчислити відношення позитивної частотності до негативної і записати в 'ratio'.



In [35]:
def get_ratio(freqs, word):
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}

    pos_neg_ratio['positive'] = lookup(freqs, word, 1)

    pos_neg_ratio['negative'] = lookup(freqs, word, 0)

    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1) / (pos_neg_ratio['negative'] + 1)

    return pos_neg_ratio

def lookup(freqs, word, label):
    n = 0

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [37]:
get_ratio(freqs, 'luck')

{'positive': 26, 'negative': 5, 'ratio': 4.5}

Функція get_words_by_threshold повертає слова, які мають відношення частотностей, більше або менше заданого порогу, залежно від мітки класу.

Алгоритм get_words_by_threshold:
- Ініціалізувати порожній словник word_list.  
- Для кожного слова у словнику частотностей:
    - Обчислити відношення частотностей з допомогою get_ratio.
    - Якщо мітка = 1 і відношення >= поріг, додати слово і його pos_neg_ratio до word_list.
    - Якщо мітка = 0 і відношення <= поріг, додати слово і його pos_neg_ratio до word_list.


In [38]:
def get_words_by_threshold(freqs, label, threshold):
    word_list = {}

    for key in freqs.keys():
        word, _ = key

        pos_neg_ratio = get_ratio(freqs, word)

        if label == 1 and pos_neg_ratio['ratio'] >= threshold:
            word_list[word] = pos_neg_ratio
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
            word_list[word] = pos_neg_ratio

    return word_list


In [47]:
get_words_by_threshold(freqs, 0, 0.2)

{'miss': {'positive': 17, 'negative': 239, 'ratio': 0.075},
 'hurt': {'positive': 4, 'negative': 33, 'ratio': 0.14705882352941177},
 'sorri': {'positive': 13, 'negative': 118, 'ratio': 0.11764705882352941},
 'cri': {'positive': 7, 'negative': 39, 'ratio': 0.2},
 'tire': {'positive': 5, 'negative': 37, 'ratio': 0.15789473684210525},
 'sadli': {'positive': 1, 'negative': 11, 'ratio': 0.16666666666666666},
 'sad': {'positive': 5, 'negative': 100, 'ratio': 0.0594059405940594},
 'ugh': {'positive': 1, 'negative': 23, 'ratio': 0.08333333333333333},
 'leed': {'positive': 1, 'negative': 9, 'ratio': 0.2},
 'sick': {'positive': 1, 'negative': 32, 'ratio': 0.06060606060606061},
 'bc': {'positive': 7, 'negative': 39, 'ratio': 0.2},
 '😭': {'positive': 2, 'negative': 23, 'ratio': 0.125},
 'disappoint': {'positive': 1, 'negative': 11, 'ratio': 0.16666666666666666},
 'poor': {'positive': 3, 'negative': 24, 'ratio': 0.16},
 '�': {'positive': 2, 'negative': 21, 'ratio': 0.13636363636363635},
 'kikchat':


8. Проаналізувати помилки класифікації.

Алгоритм:
- Для кожної пари (твіт, справжня мітка) з тестового набору:
    - Класифікувати твіт та отримати передбачену ймовірність.
    - Якщо справжня мітка не збігається з передбаченою:
        - Вивести справжню мітку, передбачену мітку і оброблений твіт.


In [48]:
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))


Truth Predicted Tweet
1	0.00	b'truli later move know queen bee upward bound movingonup'
1	0.00	b'new report talk burn calori cold work harder warm feel better weather :p'
1	0.00	b'harri niall 94 harri born ik stupid wanna chang :d'
1	0.00	b'park get sunlight'
1	0.00	b'uff itna miss karhi thi ap :p'
0	1.00	b'hello info possibl interest jonatha close join beti :( great'
0	1.00	b'u prob fun david'
0	1.00	b'pat jay'
0	1.00	b'sr financi analyst expedia inc bellevu wa financ expediajob job job hire'



9. Протестувати класифікатор на власному твіті. Розгорніть відповідну сторінку на GitHub.
10. Завантажити код та результати на GitHub, надати посилання на них та сторінку у Moodle.


In [50]:
my_tweet = 'I am happy because I am sad'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)


-0.7027547407463546


In [51]:
import json

# Передбачається, що модель вже навчена і наявні logprior та loglikelihood
data = {
    'logprior': logprior,
    'loglikelihood': loglikelihood
}

# Збереження у файлі JSON
with open('naive_bayes_model.json', 'w') as outfile:
    json.dump(data, outfile)